In [1]:
# Import Libraries
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string

In [3]:
# Read the CSV file
df = pd.read_csv('emails.csv')

#print first 5 rows
df.head(5)

,text,spam,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106,Unnamed: 107,Unnamed: 108,Unnamed: 109
0,Subject: naturally irresistible your corporate...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Subject: the stock trading gunslinger fanny i...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Subject: unbelievable new homes made easy im ...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Subject: 4 color printing special request add...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Subject: do not have money , get software cds ...",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Dropping empty columns from dataset
df=df.iloc[:,0:2]

# Printing to check if empty columns are removed
df.head(5)

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [5]:
#checking for duplicate data
df.duplicated().sum()

33

In [6]:
# Removing duplicate data from dataset
df=df.drop_duplicates()

# Checking if all duplicate data are removed 
df.duplicated().sum()

0

In [7]:
#checking if there is any missing data (ie, NaN, NAN, na) data from each column
df.isnull().sum()

text    0
spam    2
dtype: int64

In [8]:
# Removing missing data 
df.dropna(subset=['spam'],inplace=True)

# Checking if missing data is removed
df.isnull().sum()

text    0
spam    0
dtype: int64

In [9]:
# DOwnload the stopwards package
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
def process_text(text):
  #1
  nopunc = [char for char in text if char not in string.punctuation]
  nopunc = ''.join(nopunc)

  #2 
  clean_words = [ word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

  #3 
  return clean_words

In [11]:
# Show the tokenization ( a list of tokens also called lemmas)
df['text'].head().apply(process_text)

0    [Subject, naturally, irresistible, corporate, ...
1    [Subject, stock, trading, gunslinger, fanny, m...
2    [Subject, unbelievable, new, homes, made, easy...
3    [Subject, 4, color, printing, special, request...
4    [Subject, money, get, software, cds, software,...
Name: text, dtype: object

In [12]:
# Example


message4 = 'hello word hello hello world play'
message5 = 'test test test test one hello'
print(message4)
print()

# Convert the text to a matric of token counts
from sklearn.feature_extraction.text import CountVectorizer
bow4 = CountVectorizer(analyzer=process_text).fit_transform([[message4], [message5]])
print(bow4)
print()

print(bow4.shape)

hello word hello hello world play

  (0, 0)	3
  (0, 4)	1
  (0, 5)	1
  (0, 2)	1
  (1, 0)	1
  (1, 3)	4
  (1, 1)	1

(2, 6)


In [13]:
# Convert a collection of text to a matrix of tokens
from sklearn.feature_extraction.text import CountVectorizer
messages_bow =  CountVectorizer(analyzer=process_text).fit_transform(df['text'])

In [14]:
#Split the data into 80% training and 20% testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(messages_bow, df['spam'], test_size=0.20, random_state=0)

In [15]:
# Get the shape of messages_bow
messages_bow.shape

(5696, 37112)

In [16]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB().fit(X_train,y_train)

In [17]:
# Print the predictions
a=classifier.predict(X_train)
a

# Print the actual values
print(y_train.values)

['0' '0' '0' ... '0' '0' '0']


In [18]:
# Evaluate the model on the training dataset
import sklearn.metrics as metrics
pred = classifier.predict(X_test)
from sklearn.metrics import confusion_matrix , accuracy_score

print(metrics.f1_score(y_test, pred, average='weighted', labels=np.unique(pred)))
print(confusion_matrix(y_test,pred))
accuracy_score(y_test,pred)

0.9885549032775164
[[  0   0   0   0]
 [  0   0   0   0]
 [  1   0 866   7]
 [  6   1   2 257]]


0.9850877192982456

In [19]:
classifier.predict(X_test[1])

array(['1'], dtype='<U92')